In [4]:
import os
import numpy as np
from PIL import Image
TAG_FLOAT = 202021.25

In [9]:
def read_flow(file):

    assert type(file) is str, "file is not str %r" % str(file)
    assert os.path.isfile(file) is True, "file does not exist %r" % str(file)
    assert file[-4:] == '.flo', "file ending is not .flo %r" % file[-4:]
    f = open(file,'rb')
    flo_number = np.fromfile(f, np.float32, count=1)[0]
    assert flo_number == TAG_FLOAT, 'Flow number %r incorrect. Invalid .flo file' % flo_number
    w = np.fromfile(f, np.int32, count=1)
    h = np.fromfile(f, np.int32, count=1)
    #if error try: data = np.fromfile(f, np.float32, count=2*w[0]*h[0])
    data = np.fromfile(f, np.float32, count=2*w*h)
    # Reshape data into 3D array (columns, rows, bands)
    flow = np.resize(data, (int(h), int(w), 2))
    f.close()

    return flow

def transf_slow(feat_arr, flow_arr):
    feat_shape = feat_arr.shape
    flow_shape = flow_arr.shape
    assert feat_shape[0] == flow_shape[0], "Feat_map/Flow shape does not match!"
    assert feat_shape[1] == flow_shape[1], "Feat_map/Flow shape does not match!"
    h = feat_shape[0]
    w = feat_shape[1]
    new_feat = np.zeros((h, w, 3), np.uint8)
    for idx_h in range(h):
        for idx_w in range(w):
            motion_h = int(round(flow_arr[idx_h][idx_w][1]))
            motion_w = int(round(flow_arr[idx_h][idx_w][0]))
            new_h = idx_h + motion_h
            new_w = idx_w + motion_w
            if new_h < h and new_h >= 0 and new_w < w and new_w >= 0:
                    new_feat[new_h][new_w] = feat_arr[idx_h][idx_w]

    return new_feat


In [11]:
flo_file = "../../../DAVIS17_train_val/Flow/480p/breakdance/00000.flo"
img0 = "../../../DAVIS17_train_val/JPEGImages/480p/breakdance/00000.jpg"

flow_arr = read_flow(flo_file)
img_arr = np.array(Image.open(img0)).astype(np.uint8)
trans_arr = transf_slow(img_arr, flow_arr)
trans_img = Image.fromarray(trans_arr, mode='RGB')
trans_img.save('trans_img.jpg')